## Data Gathering Techniques

In [ ]:
import subprocess
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import json

pd.set_option('display.max_rows',500)

- RKI, webscrape (webscraping) https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
- John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19.git
- REST API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com/

## GITHUB CSV Data

- git clone/pull https://github.com/CSSEGISandData/COVID-19.git

In [ ]:
#To update the data
git_pull = subprocess.Popen( "git pull", 
                     cwd = os.path.dirname('C:/Users/thire/Desktop/eds_lodaya_406410_covid_19/data/raw/COVID-19/'), 
                     shell = True, 
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()


print("Error : " + str(error)) 
print("out : " + str(out))

In [ ]:
data_path=('C:/Users/thire/Desktop/eds_lodaya_406410_covid_19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
pd_raw=pd.read_csv(data_path)

In [ ]:
pd_raw

## Web Scrapping using Python

In [ ]:
page= requests.get('https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html')

In [ ]:
soup=BeautifulSoup(page.content,'html.parser')

In [ ]:
html_table=soup.find('table')

In [ ]:
all_rows=html_table.find_all('tr')

In [ ]:
final_data_list=[]

In [ ]:
for pos,rows in enumerate(all_rows):
    print(pos)
    #print(rows)
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')]
    #for each_col in rows.find_all('td'):
    final_data_list.append(col_list)

In [ ]:
pd.DataFrame(final_data_list).dropna().rename(columns={0:'state'})

## Rest API Calls

In [ ]:
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [ ]:
json_object=json.loads(data.content)

In [ ]:
type(json_object)

In [ ]:
json_object.keys()

In [ ]:
json_object

In [ ]:
full_list=[]
for pos, each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])

In [ ]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

## API acess via REST service 

- examples of a REST conform interface(attention registration mandartory

https://smartable.ai/

In [ ]:
headers = {
    'Cache-Control': 'no-cache',
    'Subscription-Key': 'c27b5022a7424d69aa05ddc325ae5184',
}

response = requests.get('https://api.smartable.ai/coronavirus/stats/US', headers=headers)
print(response)

In [ ]:
US_dict=json.loads(response.content)
with open('C:/Users/thire/Desktop/eds_lodaya_406410_covid_19/data/raw/Smartable/US_data.json', 'w') as outfile:
    json.dump(US_dict, outfile,indent=2)

## Individual States of US

In [ ]:
US_dict['stats']['breakdowns'][0]  #to understan the data in dict.

In [ ]:
# Extract data of interest from dict.

full_list_US_country=[]
for pos, each_dict in enumerate(US_dict['stats']['breakdowns'][:]):
    flatten_dict = each_dict['location']
    flatten_dict.update(dict(list(US_dict['stats']['breakdowns'][pos].items())[1:7]))
    full_list_US_country.append(flatten_dict)

In [ ]:
pd.DataFrame(full_list_US_country).to_csv('C:/Users/thire/Desktop/eds_lodaya_406410_covid_19/data/raw/Smartable/full_list_US_country.csv',sep=';',index=False)